Start with

- Formations (under center, shotgun, pistol)
- Personnel (% 11, % mult TEs, % no TEs, % mult RBs, % no RBs, % extra OL)

- % Pass
- % Pass neutral downs
- QB Scrambles

- ADOT
- % Screens
- % Long
- % passes from play-action
- % passes from under center vs shotgun vs pistol
- number receivers in top 80% targets

- % runs middle, guard/tackle, edge
- % rushes from under center vs. shotgun vs. pistol
- number rushers to account for 20% rushes

In [2]:
''' Imports '''

import pandas as pd
import polars as pl
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from prep_data import load_stats_team_tendencies_offense, load_stats_team_tendencies_defense

In [3]:
offense_tendencies = load_stats_team_tendencies_offense()

print(offense_tendencies.head().to_string())

                Games  Drives  Plays  Neutral_Down_Plays Pass_Plays  Neutral_Down_Pass Pass_Attempts QBScrambles     IAY IAY_ToSticks TotalTimeToThrow  Pass_BehindLOS  Pass_Deep Sacks Rush_Plays Rush_Attempts  Rush_Inside  Rush_Outside  Plays_11_Personnel Plays_Mult_RBs Plays_Zero_RBs Plays_Mult_TEs Plays_Zero_TEs Plays_Extra_OL  Plays / Game  Drives / Game    % Pass  % Pass Neutral Downs Scrambles / Game      ADOT ADOT to Sticks Avg Time to Throw % Passes Behind LOS % Passes Deep % Rush Inside % Rush Outside  % Plays Plays_11_Personnel % Plays Plays_Mult_RBs % Plays Plays_Zero_RBs % Plays Plays_Mult_TEs % Plays Plays_Zero_TEs % Plays Plays_Extra_OL Shotgun Plays Under Center Plays Shotgun Neutral_Down_Plays Under Center Neutral_Down_Plays Shotgun % Pass Under Center % Pass % Under Center % Shotgun % Under Center Neutral Downs % Shotgun Neutral Downs  MaxTargets  MaxTargetShare  N_Receivers_FivePctTargetShare  MaxRushAttempts  MaxRushAttemptsShare  N_Rushers_TenPctAttemptShare
posteam 

In [ ]:
''' Variance '''



In [ ]:
''' Correlation Matrix '''



# Model Preprocessing

In [ ]:
''' Transform and Scale '''


# PCA

# DBSCAN